In [59]:
import numpy as np
import pandas as pd

In [60]:
import re
import gensim
from gensim.models.word2vec import Word2Vec

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [62]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [63]:
from keras.models import Sequential
from keras.layers import Dense

In [64]:
data = pd.read_csv('/content/tweets.csv')

In [65]:
pd.set_option('display.max_colwidth' , None)

In [66]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [67]:
data.shape

(7920, 3)

In [68]:
data.drop('id',axis=1,inplace=True)

In [69]:
data.isna().sum()

label    0
tweet    0
dtype: int64

In [70]:
data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [71]:
#url removing
def remove_URL(sample):
  return re.sub(r"http\S+", "", sample)

In [72]:
data['text_clean_url-less'] = data['tweet'].apply(lambda x: remove_URL(x))

In [73]:
data['clean_tweet'] = data['text_clean_url-less'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [74]:
data.head()

,label,tweet,text_clean_url-less,clean_tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,#fingerprint #Pregnancy Test #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect...,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [75]:
data.drop(['tweet','text_clean_url-less'],axis = 1 ,inplace=True)

In [76]:
data.shape

(7920, 2)

In [77]:
x_train,x_test,y_train,y_test = train_test_split(data['clean_tweet'],data['label'],test_size=0.2,random_state=42)

In [78]:
model_gs = Word2Vec(x_train, min_count = 1)

In [79]:
model_gs.wv.most_similar('apple')

[('mac', 0.999596118927002),
 ('store', 0.9994591474533081),
 ('from', 0.9994539618492126),
 ('more', 0.9994441270828247),
 ('another', 0.9994201064109802),
 ('screen', 0.9994146227836609),
 ('ever', 0.9993608593940735),
 ('work', 0.9993183612823486),
 ('days', 0.9992135763168335),
 ('watch', 0.9992071390151978)]

In [80]:
words = model_gs.wv.index_to_key

In [81]:
len(words)

14946

In [82]:
x_train_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_train],dtype=object)
x_test_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_test],dtype=object)

In [83]:
len(x_train_vec[0][0])

100

In [84]:
x_train_vec.ndim

1

In [85]:
x_train_vec_avg = []
for v in x_train_vec:
  if v.size:
    x_train_vec_avg.append(v.mean(axis = 0))
  else:
    x_train_vec_avg.append(np.zeros(100,dtype=float))
x_test_vec_avg = []
for v in x_test_vec:
  if v.size:
    x_test_vec_avg.append(v.mean(axis = 0))
  else:
    x_test_vec_avg.append(np.zeros(100,dtype=float))

In [86]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train_vec_avg,y_train)

RandomForestClassifier()

In [87]:
#training accuracy
y_pred = clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.9993686868686869

In [88]:
y_pred = clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8446969696969697

In [89]:
svc = SVC(kernel='rbf')

In [90]:
svc.fit(x_train_vec_avg,y_train)

SVC()

In [91]:
y_pred = svc.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8421717171717171

In [92]:
y_pred = svc.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8428030303030303

In [93]:
xgb_clf = xgb.XGBClassifier(n_estimators=20 ,max_depth = 1, random_state = 42, learning_rate = 0.4, gamma = 0.1 )

In [94]:
xgb_clf.fit(x_train_vec_avg,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [95]:
y_pred = xgb_clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8388573232323232

In [96]:
y_pred = xgb_clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8446969696969697

In [97]:
knn = KNeighborsClassifier(n_neighbors = 25, metric = 'minkowski',p = 1)

In [98]:
knn.fit(x_train_vec_avg,y_train)

KNeighborsClassifier(n_neighbors=25, p=1)

In [99]:
y_pred = knn.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8541666666666666

In [100]:
y_pred = knn.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8377525252525253

In [101]:
model = Sequential()

In [102]:
model.add(Dense(85,activation = 'relu'))
model.add(Dense(80,activation = 'relu'))
model.add(Dense(65,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [103]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [104]:
x_train_new = np.array(x_train_vec_avg)
y_train_new = np.array(y_train)

In [105]:
history = model.fit(x_train_new,y_train_new,epochs=100,validation_split=0.2)

Epoch 1/100
159/159 [==============================] - 2s 4ms/step - loss: 0.4148 - accuracy: 0.7966 - val_loss: 0.3544 - val_accuracy: 0.8454
Epoch 2/100
159/159 [==============================] - 0s 3ms/step - loss: 0.3632 - accuracy: 0.8285 - val_loss: 0.3436 - val_accuracy: 0.8557
Epoch 3/100
159/159 [==============================] - 0s 3ms/step - loss: 0.3613 - accuracy: 0.8240 - val_loss: 0.3414 - val_accuracy: 0.8486
Epoch 4/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3510 - accuracy: 0.8362 - val_loss: 0.3557 - val_accuracy: 0.8344
Epoch 5/100
159/159 [==============================] - 0s 3ms/step - loss: 0.3498 - accuracy: 0.8321 - val_loss: 0.3387 - val_accuracy: 0.8525
Epoch 6/100
159/159 [==============================] - 0s 3ms/step - loss: 0.3474 - accuracy: 0.8348 - val_loss: 0.3310 - val_accuracy: 0.8549
Epoch 7/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3432 - accuracy: 0.8352 - val_loss: 0.3278 - val_accuracy: 0.8620

In [106]:
x_test_new = np.array(x_test_vec_avg)
y_test_new = np.array(y_test)

In [107]:
model.evaluate(x_test_new,y_test_new)

50/50 [==============================] - 0s 2ms/step - loss: 0.3332 - accuracy: 0.8529


[0.333225280046463, 0.8529040217399597]

In [108]:
label_dict = {1 :'Negative sentiment on the product',0 :'Positive sentiment on the product'}

In [109]:
def sentiment_analyze(tweet):
  tweet = remove_URL(tweet)
  tweet1 = gensim.utils.simple_preprocess(tweet)
  tweet_vec_avg = np.array([model_gs.wv[i] for i in tweet1 if i in words]).mean(axis = 0)
  test = tweet_vec_avg.reshape(1,100)
  sentiment = model.predict(test)
  sentiment[sentiment<0.5] = 0
  sentiment[sentiment>0.5] = 1
  print(label_dict[sentiment[0][0]])

In [110]:
tweet = 'wow!!! my apple iphone 11 is simply the best #iphonelife #love #awesome'

In [111]:
sentiment_analyze(tweet)

1/1 [==============================] - 0s 100ms/step
Positive sentiment on the product


In [112]:
tweet2 = 'wow!! apple devices have become a nightmare.stupid phone'

In [113]:
sentiment_analyze(tweet2)

1/1 [==============================] - 0s 24ms/step
Negative sentiment on the product
